<a href="https://colab.research.google.com/github/tm28823289/CSMML16_ML-traditional-deep-neural-model-comparison/blob/main/CSMML16_ML_traditional_%26_deep_neural_model_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Importing the required Python and ML libraries & modules

In [3]:
# Importing generic python modules & libraries for implementing ML models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split # For testing and training splits
from sklearn.model_selection import GridSearchCV # For searching over possible values of hyperparameters
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

# Importing modules & libraries for implementing ML traditional model
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score # For evaluating model accuracy score
from sklearn.model_selection import cross_val_score # For cross validation

# for implementing ML DNN model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [4]:
print(tf.__version__) # checking the tensorflow version

2.4.1


# 2. Loading the ATLAS data

In [ ]:
atlas_labels = pd.read_csv("C:/Users/satis/OneDrive/Desktop/ATLAS-labels.csv", header=None)
atlas_data = pd.read_csv("ATLAS-data.csv", header=None)

# 3. Exploring and experimenting with the ATLAS data

# 4. Building ML Traditional approaches

# 5. Building Deep Learning approaches

# 6. Experiment results and analysis

# 7. Comparison between Traditional and Deep Learning methods